
required tmc data : TMC_Identification.csv

In [1]:
import os
import math
import datetime
import numpy as np
import pandas as pd
import geopandas as gpd

In [2]:
def create_folder(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [3]:
def Create_Boundary(node_csv):
    df = node_csv
    gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['x_coord'], df['y_coord']))
    
    minx = gdf.bounds['minx'].min()
    miny = gdf.bounds['miny'].min()
    maxx = gdf.bounds['maxx'].max()
    maxy = gdf.bounds['maxy'].max()
    bbox = [minx,miny,maxx,maxy]
    return bbox

In [4]:
def Convert_TMC(tmc_path,gmns_path):
    create_folder(gmns_path)
    print('reading tmc data...')

    tmc = pd.read_csv(tmc_path + os.sep + 'TMC_Identification.csv')

    '''build node.csv'''
    print('converting tmc data into gmns format...')


    node_csv = pd.DataFrame()
    node_csv['name'] = None
    node_csv['x_coord'] = None
    node_csv['y_coord'] = None
    node_csv['z_coord'] = None
    node_csv['node_type'] = None
    node_csv['ctrl_type'] = None
    node_csv['zone_id'] = None
    node_csv['parent_node_id'] = None
    node_csv['geometry'] = None

    for i in range(0,len(tmc)-1):
        if tmc.loc[i+1,'road_order'] > tmc.loc[i,'road_order']:
            node_csv = node_csv.append({'name': tmc.loc[i,'tmc'],\
                                        'x_coord': tmc.loc[i,'x_coord'], \
                                        'y_coord': tmc.loc[i,'y_coord'],\
                                        'z_coord': None,\
                                        'node_type': 'tmc_start',\
                                        'ctrl_type': None,\
                                        'zone_id': None,\
                                        'parent_node_id': None,\
                                        'geometry': "POINT (" + tmc.loc[i,'x_coord'].astype(str) + " " + tmc.loc[i,'y_coord'].astype(str) +")"}, ignore_index=True)
        else:
            node_csv = node_csv.append({'name': tmc.loc[i,'tmc'],\
                                        'x_coord': tmc.loc[i,'x_coord'], \
                                        'y_coord': tmc.loc[i,'y_coord'],\
                                        'z_coord': None,\
                                        'node_type': 'tmc_start',\
                                        'ctrl_type': None,\
                                        'zone_id': None,\
                                        'parent_node_id': None,\
                                        'geometry': "POINT (" + tmc.loc[i,'x_coord'].astype(str) + " " + tmc.loc[i,'y_coord'].astype(str) +")"}, ignore_index=True)
            node_csv = node_csv.append({'name': tmc.loc[i,'tmc']+'END',\
                                        'x_coord': tmc.loc[i,'end_longitude'], \
                                        'y_coord': tmc.loc[i,'end_latitude'],\
                                        'z_coord': None,\
                                        'node_type': 'tmc_end',\
                                        'ctrl_type': None,\
                                        'zone_id': None,\
                                        'parent_node_id': None,\
                                        'geometry': "POINT (" + tmc.loc[i,'end_longitude'].astype(str) + " " + tmc.loc[i,'end_latitude'].astype(str) +")"}, ignore_index=True)


    node_csv = node_csv.append({'name': tmc.loc[i+1,'tmc'],\
                                        'x_coord': tmc.loc[i+1,'x_coord'], \
                                        'y_coord': tmc.loc[i+1,'y_coord'],\
                                        'z_coord': None,\
                                        'node_type': 'tmc_start',\
                                        'ctrl_type': None,\
                                        'zone_id': None,\
                                        'parent_node_id': None,\
                                        'geometry': "POINT (" + tmc.loc[i+1,'x_coord'].astype(str) + " " + tmc.loc[i+1,'y_coord'].astype(str) +")"}, ignore_index=True)

    node_csv = node_csv.append({'name': tmc.loc[i+1,'tmc']+'END',\
                                        'x_coord': tmc.loc[i+1,'end_longitude'], \
                                        'y_coord': tmc.loc[i+1,'end_latitude'],\
                                        'z_coord': None,\
                                        'node_type': 'tmc_end',\
                                        'ctrl_type': None,\
                                        'zone_id': None,\
                                        'parent_node_id': None,\
                                        'geometry': "POINT (" + tmc.loc[i+1,'end_longitude'].astype(str) + " " + tmc.loc[i+1,'end_latitude'].astype(str) +")"}, ignore_index=True)

    node_csv.index.name = 'node_id'

    node_csv.index += 100000001 #index from 0

    node_csv.to_csv(gmns_path + os.sep + '/node_tmc.csv')


    '''build link.csv'''
    link_csv = pd.DataFrame()
    link_csv['name'] = None
    link_csv['from_node_id'] = None
    link_csv['to_node_id'] = None
    link_csv['directed'] = None
    link_csv['geometry_id'] = None
    link_csv['geometry'] = None
    link_csv['dir_flag'] = None
    link_csv['parent_link_id'] = None
    link_csv['length'] = None
    link_csv['grade'] = None
    link_csv['facility_type'] = None
    link_csv['capacity'] = None
    link_csv['free_speed'] = None
    link_csv['lanes'] = None

    for i in range(0,len(tmc)):
        link_csv = link_csv.append({'name': tmc.loc[i,'tmc'],\
                                        'from_node_id': node_csv[(node_csv['x_coord']==tmc.loc[i,'x_coord']) & (node_csv['y_coord']==tmc.loc[i,'y_coord'])].index.values[0], \
                                        'to_node_id': node_csv[(node_csv['x_coord']==tmc.loc[i,'end_longitude']) & (node_csv['y_coord']==tmc.loc[i,'end_latitude'])].index.values[0],\
                                        'directed': 1,\
                                        'geometry_id': None,\
                                        'geometry': "LINESTRING (" + tmc.loc[i,'x_coord'].astype(str) + " " + tmc.loc[i,'y_coord'].astype(str) + "," +\
                                            tmc.loc[i,'end_longitude'].astype(str) + tmc.loc[i,'end_latitude'].astype(str) + ")",\
                                        'dir_flag': 1,\
                                        'parent_link_id': None,\
                                        'length': tmc.loc[i,'miles'],\
                                        'grade': None,\
                                        'facility_type': 'interstate' if tmc.loc[i,'road'][0] == 'I'else None ,\
                                        'capacity':None,\
                                        'free_speed':None,\
                                        'lanes': None}, ignore_index=True)
    link_csv.index.name = 'link_id'
    link_csv.index += 100000001

    link_csv.to_csv(gmns_path + os.sep + '/link_tmc.csv')

    return node_csv, link_csv

In [ ]:
"Step 0.1 Convert TMC Data into GMNS Format"

tmc_path = '/usr/local/home/ysx28/Desktop/GMNS/TMC2GMNS/TMC'
gmns_path = '/usr/local/home/ysx28/Desktop/GMNS/TMC2GMNS/GMNS'

import tmc2gmns as gg
node_transit,link_transit = gg.Convert_TMC(tmc_path,gmns_path)


In [1]:
"Step 0.2 Get the OSM Network"

import os
import osm2gmns as og

osm_path = 'OSM/consolidated'

net = og.getNetFromOSMFile('OSM/tmc.osm',network_type=('auto'), default_lanes=True, default_speed=True)
og.consolidateComplexIntersections(net)
og.outputNetToCSV(net, output_folder=osm_path)

# net = og.getNetFromCSV(osm_path)

# og.outputNetToCSV(net, output_folder=osm_path)

osm2gmns, version 0.4.2


In [ ]:
"Step 1 Create the Connector"

import tmc2gmns as gg
node,link_osm_connector = gg.CreatConnector_osm_tmc(osm_path,gmns_path)

In [32]:
create_folder(gmns_path)
print('reading tmc data...')

tmc = pd.read_csv(tmc_path + os.sep + 'TMC_Identification.csv')
# tmc = tmc.sort_values(by=['road_order'])

'''build node.csv'''
print('converting tmc data into gmns format...')


node_csv = pd.DataFrame()
node_csv['name'] = None
node_csv['x_coord'] = None
node_csv['y_coord'] = None
node_csv['z_coord'] = None
node_csv['node_type'] = None
node_csv['ctrl_type'] = None
node_csv['zone_id'] = None
node_csv['parent_node_id'] = None
node_csv['geometry'] = None

for i in range(0,len(tmc)-1):
    if tmc.loc[i+1,'road_order'] > tmc.loc[i,'road_order']:
        node_csv = node_csv.append({'name': tmc.loc[i,'tmc'],\
                                    'x_coord': tmc.loc[i,'x_coord'], \
                                    'y_coord': tmc.loc[i,'y_coord'],\
                                    'z_coord': None,\
                                    'node_type': 'tmc_start',\
                                    'ctrl_type': None,\
                                    'zone_id': None,\
                                    'parent_node_id': None,\
                                    'geometry': "POINT (" + tmc.loc[i,'x_coord'].astype(str) + " " + tmc.loc[i,'y_coord'].astype(str) +")"}, ignore_index=True)
    else:
        node_csv = node_csv.append({'name': tmc.loc[i,'tmc'],\
                                    'x_coord': tmc.loc[i,'x_coord'], \
                                    'y_coord': tmc.loc[i,'y_coord'],\
                                    'z_coord': None,\
                                    'node_type': 'tmc_start',\
                                    'ctrl_type': None,\
                                    'zone_id': None,\
                                    'parent_node_id': None,\
                                    'geometry': "POINT (" + tmc.loc[i,'x_coord'].astype(str) + " " + tmc.loc[i,'y_coord'].astype(str) +")"}, ignore_index=True)
        node_csv = node_csv.append({'name': tmc.loc[i,'tmc']+'END',\
                                    'x_coord': tmc.loc[i,'end_longitude'], \
                                    'y_coord': tmc.loc[i,'end_latitude'],\
                                    'z_coord': None,\
                                    'node_type': 'tmc_end',\
                                    'ctrl_type': None,\
                                    'zone_id': None,\
                                    'parent_node_id': None,\
                                    'geometry': "POINT (" + tmc.loc[i,'end_longitude'].astype(str) + " " + tmc.loc[i,'end_latitude'].astype(str) +")"}, ignore_index=True)


node_csv = node_csv.append({'name': tmc.loc[i+1,'tmc'],\
                                    'x_coord': tmc.loc[i+1,'x_coord'], \
                                    'y_coord': tmc.loc[i+1,'y_coord'],\
                                    'z_coord': None,\
                                    'node_type': 'tmc_start',\
                                    'ctrl_type': None,\
                                    'zone_id': None,\
                                    'parent_node_id': None,\
                                    'geometry': "POINT (" + tmc.loc[i+1,'x_coord'].astype(str) + " " + tmc.loc[i+1,'y_coord'].astype(str) +")"}, ignore_index=True)

node_csv = node_csv.append({'name': tmc.loc[i+1,'tmc']+'END',\
                                    'x_coord': tmc.loc[i+1,'end_longitude'], \
                                    'y_coord': tmc.loc[i+1,'end_latitude'],\
                                    'z_coord': None,\
                                    'node_type': 'tmc_end',\
                                    'ctrl_type': None,\
                                    'zone_id': None,\
                                    'parent_node_id': None,\
                                    'geometry': "POINT (" + tmc.loc[i+1,'end_longitude'].astype(str) + " " + tmc.loc[i+1,'end_latitude'].astype(str) +")"}, ignore_index=True)

node_csv.index.name = 'node_id'

node_csv.index += 100000001 #index from 0

node_csv.to_csv(gmns_path + os.sep + '/node_tmc.csv')

reading tmc data...
converting tmc data into gmns format...


In [33]:
'''build link.csv'''
link_csv = pd.DataFrame()
link_csv['name'] = None
link_csv['from_node_id'] = None
link_csv['to_node_id'] = None
link_csv['directed'] = None
link_csv['geometry_id'] = None
link_csv['geometry'] = None
link_csv['dir_flag'] = None
link_csv['parent_link_id'] = None
link_csv['length'] = None
link_csv['grade'] = None
link_csv['facility_type'] = None
link_csv['capacity'] = None
link_csv['free_speed'] = None
link_csv['lanes'] = None

for i in range(0,len(tmc)):
    link_csv = link_csv.append({'name': tmc.loc[i,'tmc'],\
                                    'from_node_id': node_csv[(node_csv['x_coord']==tmc.loc[i,'x_coord']) & (node_csv['y_coord']==tmc.loc[i,'y_coord'])].index.values[0], \
                                    'to_node_id': node_csv[(node_csv['x_coord']==tmc.loc[i,'end_longitude']) & (node_csv['y_coord']==tmc.loc[i,'end_latitude'])].index.values[0],\
                                    'directed': 1,\
                                    'geometry_id': None,\
                                    'geometry': "LINESTRING (" + tmc.loc[i,'x_coord'].astype(str) + " " + tmc.loc[i,'y_coord'].astype(str) + "," +\
                                        tmc.loc[i,'end_longitude'].astype(str) + tmc.loc[i,'end_latitude'].astype(str) + ")",\
                                    'dir_flag': 1,\
                                    'parent_link_id': None,\
                                    'length': tmc.loc[i,'miles'],\
                                    'grade': None,\
                                    'facility_type': 'interstate' if tmc.loc[i,'road'][0] == 'I'else None ,\
                                    'capacity':None,\
                                    'free_speed':None,\
                                    'lanes': None}, ignore_index=True)
link_csv.index.name = 'link_id'
link_csv.index += 100000001


link_csv.to_csv(gmns_path + os.sep + '/link_tmc.csv')


In [34]:
link_csv

,name,from_node_id,to_node_id,directed,geometry_id,geometry,dir_flag,parent_link_id,length,grade,facility_type,capacity,free_speed,lanes
link_id,,,,,,,,,,,,,,
100000001,110P04102,100000001,100000002,1,None,"LINESTRING (-77.10289 39.01887,-77.1050739.021...",1,None,0.190071,None,interstate,None,None,None
100000002,110+04103,100000002,100000003,1,None,"LINESTRING (-77.10507 39.021029999999996,-77.1...",1,None,1.215464,None,interstate,None,None,None
100000003,110P04103,100000003,100000004,1,None,"LINESTRING (-77.12297 39.03014,-77.12763000000...",1,None,0.256706,None,interstate,None,None,None
100000004,110+04104,100000004,100000005,1,None,"LINESTRING (-77.12763000000001 39.03093,-77.13...",1,None,0.563975,None,interstate,None,None,None
100000005,110P04104,100000005,100000006,1,None,"LINESTRING (-77.13789 39.03257,-77.1453539.0378)",1,None,0.560561,None,interstate,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100000066,110N04104,100000067,100000068,1,None,"LINESTRING (-77.14494 39.0363,-77.1373939.03233)",1,None,0.533999,None,interstate,None,None,None
100000067,110-04103,100000068,100000069,1,None,"LINESTRING (-77.13739 39.03233,-77.12834000000...",1,None,0.497398,None,interstate,None,None,None
100000068,110N04103,100000069,100000070,1,None,"LINESTRING (-77.12834000000001 39.03089,-77.12...",1,None,0.269579,None,interstate,None,None,None


In [35]:
tmc

,tmc,road,direction,intersection,state,county,zip,y_coord,x_coord,end_latitude,end_longitude,miles,road_order,timezone_name,type,country,active_start_date,active_end_date
0,110P04102,I-270,NORTHBOUND,I-495/MD-355,MD,MONTGOMERY,20814,39.01887,-77.10289,39.02103,-77.10507,0.190071,1,America/New_York,P1.1,USA,2019-10-31 15:30:00-04:00,NaN
1,110+04103,I-270,NORTHBOUND,MD-187/OLD GEORGETOWN RD/EXIT 1,MD,MONTGOMERY,20814,39.02103,-77.10507,39.03014,-77.12297,1.215464,2,America/New_York,P1.3,USA,2019-10-31 15:30:00-04:00,NaN
2,110P04103,I-270,NORTHBOUND,MD-187/OLD GEORGETOWN RD/EXIT 1,MD,MONTGOMERY,20814,39.03014,-77.12297,39.03093,-77.12763,0.256706,3,America/New_York,P1.3,USA,2019-10-31 15:30:00-04:00,NaN
3,110+04104,I-270,NORTHBOUND,I-270-SPUR,MD,MONTGOMERY,20817,39.03093,-77.12763,39.03257,-77.13789,0.563975,4,America/New_York,P1.1,USA,2019-10-31 15:30:00-04:00,NaN
4,110P04104,I-270,NORTHBOUND,I-270-SPUR,MD,MONTGOMERY,20817,39.03257,-77.13789,39.03780,-77.14535,0.560561,5,America/New_York,P1.1,USA,2019-10-31 15:30:00-04:00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,110N04104,I-270,SOUTHBOUND,I-270-SPUR,MD,MONTGOMERY,20817,39.03630,-77.14494,39.03233,-77.13739,0.533999,31,America/New_York,P1.1,USA,2019-10-31 15:30:00-04:00,NaN
66,110-04103,I-270,SOUTHBOUND,MD-187/OLD GEORGETOWN RD/EXIT 1,MD,MONTGOMERY,20814,39.03233,-77.13739,39.03089,-77.12834,0.497398,32,America/New_York,P1.3,USA,2019-10-31 15:30:00-04:00,NaN
67,110N04103,I-270,SOUTHBOUND,MD-187/OLD GEORGETOWN RD/EXIT 1,MD,MONTGOMERY,20814,39.03089,-77.12834,39.03001,-77.12346,0.269579,33,America/New_York,P1.3,USA,2019-10-31 15:30:00-04:00,NaN
68,110-04102,I-270,SOUTHBOUND,I-495/MD-355,MD,MONTGOMERY,20814,39.03001,-77.12346,39.02148,-77.10605,1.176331,34,America/New_York,P1.1,USA,2019-10-31 15:30:00-04:00,NaN


In [36]:
'''build link.csv'''
node_csv


,name,x_coord,y_coord,z_coord,node_type,ctrl_type,zone_id,parent_node_id,geometry
node_id,,,,,,,,,
100000001,110P04102,-77.10289,39.01887,None,tmc_start,None,None,None,POINT (-77.10289 39.01887)
100000002,110+04103,-77.10507,39.02103,None,tmc_start,None,None,None,POINT (-77.10507 39.021029999999996)
100000003,110P04103,-77.12297,39.03014,None,tmc_start,None,None,None,POINT (-77.12297 39.03014)
100000004,110+04104,-77.12763,39.03093,None,tmc_start,None,None,None,POINT (-77.12763000000001 39.03093)
100000005,110P04104,-77.13789,39.03257,None,tmc_start,None,None,None,POINT (-77.13789 39.03257)
...,...,...,...,...,...,...,...,...,...
100000068,110-04103,-77.13739,39.03233,None,tmc_start,None,None,None,POINT (-77.13739 39.03233)
100000069,110N04103,-77.12834,39.03089,None,tmc_start,None,None,None,POINT (-77.12834000000001 39.03089)
100000070,110-04102,-77.12346,39.03001,None,tmc_start,None,None,None,POINT (-77.12346 39.03001)


In [37]:
node_road

,name,node_id,osm_node_id,osm_highway,zone_id,ctrl_type,node_type,activity_type,is_boundary,x_coord,y_coord,main_node_id,poi_id,notes
0,NaN,0,281233,NaN,NaN,0,NaN,NaN,NaN,-77.142931,39.021258,NaN,NaN,NaN
1,NaN,1,281236,NaN,NaN,0,NaN,NaN,NaN,-77.141685,39.024681,NaN,NaN,NaN
2,NaN,2,281247,NaN,NaN,0,NaN,NaN,NaN,-77.144647,39.036611,NaN,NaN,NaN
3,NaN,3,281260,motorway_junction,NaN,0,NaN,NaN,NaN,-77.150676,39.047928,NaN,NaN,NaN
4,NaN,4,281305,NaN,NaN,0,NaN,NaN,NaN,-77.212601,39.140788,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27794,NaN,28595,2925340952;2925340953;2925340954;2925340955,traffic_signals,NaN,1,NaN,NaN,NaN,-77.410458,39.373454,110.0,NaN,NaN
27795,NaN,28596,3667972301;3667972302;3667972308;3667972309,traffic_signals,NaN,1,NaN,NaN,NaN,-77.153278,39.091918,0.0,NaN,NaN
27796,NaN,28597,3856249749;3856251957;5078766805;5078766806,traffic_signals,NaN,1,NaN,NaN,NaN,-77.199834,39.090659,118.0,NaN,NaN
27797,NaN,28598,4452680077;5332897470,traffic_signals,NaN,1,NaN,NaN,NaN,-77.203871,39.115793,288.0,NaN,NaN


In [45]:
import os
import math
import heapq
import numpy as np
import pandas as pd


# WGS84 transfer coordinate system to distance: meter
def LLs2Dist(lon1, lat1, lon2, lat2):
    R = 6371
    dLat = (lat2 - lat1) * math.pi / 180.0
    dLon = (lon2 - lon1) * math.pi / 180.0
    a = math.sin(dLat / 2) * math.sin(dLat/2) + math.cos(lat1 * math.pi / 180.0) * math.cos(lat2 * math.pi / 180.0) * math.sin(dLon/2) * math.sin(dLon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    dist = R * c * 1000
    return dist


def createConnector(dataList,from_node_id, to_node_id):
    link = []
    from_node_id_x = dataList[from_node_id]['x_coord']
    from_node_id_y = dataList[from_node_id]['y_coord']
    to_node_id_x = dataList[to_node_id]['x_coord']
    to_node_id_y = dataList[to_node_id]['y_coord']
    length = LLs2Dist(from_node_id_x,from_node_id_y,to_node_id_x,to_node_id_y)
    geometry = 'LINESTRING (' + str(from_node_id_x)+' '+str(from_node_id_y)+', '+str(to_node_id_x)+' '+str(to_node_id_y)+')'
        
    link = [from_node_id, to_node_id, length, geometry]
    return link




print('creating connector between osm network and gtfs data...')

node_tmc = pd.read_csv(gmns_path + os.sep +'/node_tmc.csv',low_memory=False) # transit node
node_road = pd.read_csv(osm_path + os.sep +'/node.csv', encoding='gbk',low_memory=False)

node_combine = pd.DataFrame()
node_combine['node_id']= node_road['node_id'].tolist() + node_tmc['node_id'].tolist()
node_combine['name']= node_road['name'].tolist() + node_tmc['name'].tolist()
node_combine['x_coord']= node_road['x_coord'].tolist() + node_tmc['x_coord'].tolist()
node_combine['y_coord'] = node_road['y_coord'].tolist() + node_tmc['y_coord'].tolist()
node_combine['node_type']= node_road['node_type'].tolist() + node_tmc['node_type'].tolist()


node_combine.to_csv(gmns_path + os.sep +'/node_combine.csv', index=False)
    
link_road = pd.read_csv(osm_path + os.sep +'/link.csv', encoding='gbk',low_memory=False)
link_road = link_road.drop(['link_id', 'osm_way_id', 'from_biway'], axis=1)
    
dataList_tmc = {}
gp = node_tmc.groupby('node_id')
for key, form in gp:
    dataList_tmc[key] = {
        'x_coord': form['x_coord'].values[0],
        'y_coord': form['y_coord'].values[0]
        }

dataList_road = {}
gp = node_road.groupby('node_id')
for key, form in gp:
    dataList_road[key] = {
        'x_coord': form['x_coord'].values[0],
        'y_coord': form['y_coord'].values[0]
        }

coord_list = []
for key in dataList_road.keys(): 
    coord_list.append((dataList_road[key]['x_coord'],dataList_road[key]['y_coord']))
coord_array = np.array(coord_list)


dataList = {}  #road node+tmc node
gp = node_combine.groupby('node_id')
for key, form in gp:
    dataList[key] = {
        'x_coord': form['x_coord'].values[0],
        'y_coord': form['y_coord'].values[0]
        }

# print('building connector...')
link_list = []

for key in dataList_tmc.keys(): 
    coord = np.array((dataList_tmc[key]['x_coord'],dataList_tmc[key]['y_coord']))
    coord_diff = coord_array - coord
    coord_diff_square = np.power(coord_diff,2)
    coord_diff_sum_square = coord_diff_square.sum(axis=1)
    distance = np.sqrt(coord_diff_sum_square)  #distance array between one tmc node to all road node 
    
    count = 1
    while (count):
        idx_temp = heapq.nsmallest(count, distance.tolist())
        idx = np.where(distance == idx_temp[count-1])  #index of the nearest road node
        active_node = node_road['node_id'].iloc[idx[0]]  #id of the nearest road node
        if ((active_node[idx[0][0]] in link_road['from_node_id'].tolist()) and (active_node[idx[0][0]] in link_road['to_node_id'].tolist())): #check the road node is the from/to node of road link 
            active_link1 = createConnector(dataList,active_node[idx[0][0]], key) #create link between the nearest road node and the selected tmc node
            active_link2= createConnector(dataList,key, active_node[idx[0][0]])
            link_list.append(active_link1)
            link_list.append(active_link2)
            break
        else:
            count += 1


connector_csv = pd.DataFrame()
connector_csv = pd.DataFrame(link_list, columns=['from_node_id','to_node_id','length','geometry']).drop_duplicates()    


connector_csv['name'] = None
connector_csv['link_type'] = None
connector_csv['link_type_name'] = 'connector'
connector_csv['dir_flag'] = 1
connector_csv['lanes'] = 1
connector_csv['free_speed'] = None
connector_csv['capacity'] = None
connector_csv['allowed_uses'] = None



combined_link = pd.concat([link_road,connector_csv], axis=0, ignore_index=True)

combined_link.index.name = 'link_id'
combined_link.index += 1

combined_link.to_csv(gmns_path + os.sep +'/link_osm_connector.csv')



creating connector between osm network and gtfs data...


In [ ]:
# def CreatConnector_osm_gtfs(osm_path,gmns_path):
    
#     return node_combine, combined_link
    

    

In [ ]:
import os
import ctypes
import numpy as np
import pandas as pd
from sys import platform

# some functions --> credit to path4gmns   
def _optimal_label_correcting_CAPI(node_size,
                        from_node_no_array,
                        to_node_no_array,
                        first_link_from,
                        last_link_from,
                        sorted_link_no_array, 
                        link_cost_array,
                        node_label_cost,
                        node_predecessor,
                        link_predecessor,
                        queue_next, from_node_id,
                        internal_node_seq_no_dict, _cdll):
    """ input : origin_node,destination_node,departure_time
        output : the shortest path
    """
    origin_node_no = internal_node_seq_no_dict[from_node_id]

    _cdll.shortest_path(origin_node_no,
                        node_size,
                        from_node_no_array,
                        to_node_no_array,
                        first_link_from,
                        last_link_from,
                        sorted_link_no_array, 
                        link_cost_array,
                        node_label_cost,
                        node_predecessor,
                        link_predecessor,
                        queue_next)


def output_path_sequence(internal_node_seq_no_dict, node_predecessor, external_node_id_dict, link_predecessor, from_node_id, to_node_id):
    """ output shortest path in terms of node sequence or link sequence
    
    Note that this function returns GENERATOR rather than list.
    """
    path = []
    current_node_seq_no = internal_node_seq_no_dict[to_node_id]

    while current_node_seq_no >= 0:  
        path.append(current_node_seq_no)
        current_node_seq_no = node_predecessor[current_node_seq_no]
        # reverse the sequence
    for node_seq_no in reversed(path):
        yield external_node_id_dict[node_seq_no]


def find_shortest_path(node_size,
                       from_node_no_array,
                       to_node_no_array,
                       first_link_from,
                       last_link_from,
                       sorted_link_no_array, 
                       link_cost_array,
                       node_label_cost,
                       node_predecessor,
                       link_predecessor,
                       queue_next, internal_node_seq_no_dict, from_node_id, to_node_id, external_node_id_dict, _cdll):
    if from_node_id not in internal_node_seq_no_dict.keys():
        raise Exception(f"Node ID: {from_node_id} not in the network")
    if to_node_id not in internal_node_seq_no_dict.keys():
        raise Exception(f"Node ID: {to_node_id} not in the network")

    _optimal_label_correcting_CAPI(node_size,
                                   from_node_no_array,
                                   to_node_no_array,
                                   first_link_from,
                                   last_link_from,
                                   sorted_link_no_array, 
                                   link_cost_array,
                                   node_label_cost,
                                   node_predecessor,
                                   link_predecessor,
                                   queue_next, from_node_id, internal_node_seq_no_dict, _cdll)

    return list(output_path_sequence(internal_node_seq_no_dict, node_predecessor, external_node_id_dict, link_predecessor, from_node_id, to_node_id))



def shortest_path(node_size, link_size, from_node_no_array, to_node_no_array, from_node_id, to_node_id,
                  link_road, internal_node_seq_no_dict, external_node_id_dict, _cdll):
    
    ### allocate
    node_predecessor = np.full(node_size, -1, np.int32)
    link_predecessor = np.full(node_size, -1, np.int32)

    # initialize others as numpy arrays directly
    queue_next = np.full(node_size, 0, np.int32)
    first_link_from = np.full(node_size, -1, np.int32)
    last_link_from = np.full(node_size, -1, np.int32)
    sorted_link_no_array = np.full(link_size, -1,np.int32)
    
    
    # count the size of outgoing links for each node
    outgoing_link_list = [0] * node_size
    for link in range(link_size):
        outgoing_link_list[from_node_no_array[link]] += 1
    
    cumulative_count = 0
    for i in range(node_size):
        first_link_from[i] = cumulative_count
        last_link_from[i] = (
            first_link_from[i] + outgoing_link_list[i]
        )
        cumulative_count += outgoing_link_list[i]
    
    # reset the counter # need to construct sorted_link_no_vector
    # we are converting a 2 dimensional dynamic array to a fixed size 
    # one-dimisonal array, with the link size 
    for i in range(node_size):
        outgoing_link_list[i] = 0
    
    # count again the current size of outgoing links for each node
    for j in range(link_size):
        # fetch the curent from node seq no of this link
        from_node_seq_no = from_node_no_array[j]
        # j is the link sequence no in the original link block
        k = (first_link_from[from_node_seq_no] 
             + outgoing_link_list[from_node_seq_no])
        sorted_link_no_array[k] = j
        # continue to count, increase by 1
        outgoing_link_list[from_node_no_array[j]] += 1
       
    MAX_LABEL_COST = 10000
    link_cost = link_road['length'].tolist()
    link_cost_array = np.array(link_cost, np.float64)
    node_label_cost = np.full(node_size, MAX_LABEL_COST,np.float64)
    
    shortest_path_node_sequence_result = find_shortest_path(node_size,
                                                            from_node_no_array,
                                                            to_node_no_array,
                                                            first_link_from,
                                                            last_link_from,
                                                            sorted_link_no_array, 
                                                            link_cost_array,
                                                            node_label_cost,
                                                            node_predecessor,
                                                            link_predecessor,
                                                            queue_next, internal_node_seq_no_dict, from_node_id, to_node_id,
                                                            external_node_id_dict, _cdll)
    
    return  shortest_path_node_sequence_result
    

def Create_TransitRoute(gmns_path):
    
    print('creating transit routes...')
    # print('reading gmns data...')
    link_transit = pd.read_csv(gmns_path + os.sep +'/link_transit.csv')
    
    node_combine = pd.read_csv(gmns_path + os.sep +'/node.csv', encoding='gbk')
    link_road = pd.read_csv(gmns_path + os.sep +'/link_osm_connector.csv',low_memory=False) 
    
    
    # print('building node id and node index dict...')
    node_size = len(node_combine) # transit node and osm node
    link_size = len(link_road) # osm link and connector link
    
    # node_list = node_combine['node_id']
    # link_list = link_road['link_id'].tolist()
    internal_node_seq_no_dict = {} # node id --> node index
    external_node_id_dict = {} # node index --> node id
    
    node_combine['node_id'] = node_combine.copy()['node_id'].astype(float)
    external_node_id = node_combine.copy()['node_id']
    node_seq_no = 0
    for i in range(node_size):
        internal_node_seq_no_dict[external_node_id[i]] = node_seq_no
        external_node_id_dict[node_seq_no] = external_node_id[i]
        node_seq_no += 1
    
    # prepare
    from_node_no_array = []
    to_node_no_array = []
    
    for i in range(len(link_road['from_node_id'])):
        a = internal_node_seq_no_dict[link_road.copy()['from_node_id'].iloc[i]]
        from_node_no_array.append(a)
    
    for i in range(len(link_road['to_node_id'])):
        a = internal_node_seq_no_dict[link_road.copy()['to_node_id'].iloc[i]]
        to_node_no_array.append(a)
    
    from_node_no_array = np.array(from_node_no_array, np.int32)
    to_node_no_array = np.array(to_node_no_array, np.int32)
    
    
    
    # print('shortest path cdll initializing...')
    
    # get cdll
    _pkg_path = os.path.abspath(__file__)
    
    if platform.startswith('win32'):
        _dll_file = os.path.join(os.path.dirname(_pkg_path), './bin/path_engine.dll')
    elif platform.startswith('linux'):
        _dll_file = os.path.join(os.path.dirname(_pkg_path), './bin/path_engine.so')
    elif platform.startswith('darwin'):
        _dll_file = os.path.join(os.path.dirname(_pkg_path), './bin/path_engine.dylib')
    else:
        raise Exception('Please build the shared library compatible to your OS\
                        using source files in engine_cpp!')
    
    _cdll = ctypes.cdll.LoadLibrary(_dll_file)
    
    # set up the argument types for the shortest path function in dll.
    _cdll.shortest_path.argtypes = [
        ctypes.c_int, 
        ctypes.c_int, 
        np.ctypeslib.ndpointer(dtype=np.int32),
        np.ctypeslib.ndpointer(dtype=np.int32),
        np.ctypeslib.ndpointer(dtype=np.int32),
        np.ctypeslib.ndpointer(dtype=np.int32),
        np.ctypeslib.ndpointer(dtype=np.int32), 
        np.ctypeslib.ndpointer(dtype=np.float64),   
        np.ctypeslib.ndpointer(dtype=np.float64),                                    
        np.ctypeslib.ndpointer(dtype=np.int32),
        np.ctypeslib.ndpointer(dtype=np.int32),
        np.ctypeslib.ndpointer(dtype=np.int32),
    ]
    
    
    # a_temp = shortest_path(node_size, link_size, from_node_no_array, to_node_no_array, from_node_id, to_node_id)
    from_node_temp = np.array(link_road['from_node_id'])
    to_node_temp = np.array(link_road['to_node_id'])
    
    length_temp = np.array(link_road['length'])
    
    node_dict_x = dict(zip(node_combine['node_id'],node_combine['x_coord']))
    node_dict_y = dict(zip(node_combine['node_id'],node_combine['y_coord']))
    
    # print('finding shortest path for each transit route...')
    with open('log.txt', 'w') as f:
        f.write('Cannot find shortest path [from_node_id,to_node_id]...'+ '\n')
            
    
    for i in range(len(link_transit['link_id'])):
        from_node_id = link_transit.copy()['from_node_id'].iloc[i]
        to_node_id = link_transit.copy()['to_node_id'].iloc[i]
        
        active_shortest_node_sequence = []
        active_length_list =[]
        
        active_shortest_node_sequence = shortest_path(node_size, link_size, from_node_no_array, to_node_no_array, from_node_id, to_node_id,
                                                      link_road, internal_node_seq_no_dict, external_node_id_dict, _cdll)
        if len(active_shortest_node_sequence) == 1:
           
            with open('log.txt', 'a') as f:
                temp = [str(from_node_id),str(to_node_id)]
                f.write(','.join(temp) + '\n')
            # print('Cannot find shortest path...',from_node_id,'-->',to_node_id)
            continue
        
        
        for j in range(len(active_shortest_node_sequence)-1):
            active_from_node_id = active_shortest_node_sequence[j]
            active_to_node_id = active_shortest_node_sequence[j+1]
            temp1 = np.array(from_node_temp == active_from_node_id)
            temp2 = np.array(to_node_temp == active_to_node_id)
            temp = temp1 & temp2
            if not any(temp):
                # print('Cannot find length...', active_from_node_id, '-->', active_to_node_id)
                break
            
            active_length = length_temp[temp]
            active_length = active_length[0]
            active_length_list.append(active_length)
            
        link_transit['length'].iloc[i] = sum(active_length_list)
        
        geometry = ''
        for j in range(len(active_shortest_node_sequence)):
            if (j == len(active_shortest_node_sequence)-1):
                geometry = geometry + str(node_dict_x[active_shortest_node_sequence[j]]) + ' ' + str(node_dict_y[active_shortest_node_sequence[j]])
            else:
                geometry = geometry + str(node_dict_x[active_shortest_node_sequence[j]]) + ' ' + str(node_dict_y[active_shortest_node_sequence[j]]) + ', '
            
        geometry = 'LINESTRING (' + geometry+ ')'
        link_transit['geometry'].iloc[i] = geometry
    
    link_transit.to_csv(gmns_path + os.sep +'/link_transit.csv', index=False)
    
    combined_link = pd.concat([link_road,link_transit], axis=0, ignore_index=True)
    combined_link.to_csv(gmns_path + os.sep +'/link.csv', index=False)
    return combined_link

In [8]:
from math import sin, cos, sqrt, atan2, radians
def distance_cal(lat1,lon1,lat2,lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [11]:
distance_dict = {}
# pd.DataFrame([])
for i in range(len(node)):
    node_tmc_dist = []
    for j in range(len(tmc)):
         node_tmc_dist.append(distance_cal(node['y_coord'][i],node['x_coord'][i],tmc['end_latitude'][j],tmc['end_longitude'][j]))
    distance_dict[node['node_id'][i]] = node_tmc_dist

    
df_dist=pd.DataFrame.from_dict(distance_dict)

In [38]:
# df_dist[0].
df_dist.nsmallest(5, 0)

,0,1,2,3,4,5,6,7,8,9,...,5853,5854,5855,5856,5857,5858,5859,5860,5861,5862
34,3146.511875,3153.590014,3147.343112,3147.391434,3151.437944,3151.109146,3150.900845,3150.283570,3147.641422,3149.786815,...,3149.288685,3148.065627,3150.256717,3148.797131,3149.942662,3149.928059,3150.005094,3152.691612,3149.380895,3149.054704
35,3146.797636,3153.875954,3147.628761,3147.677070,3151.723963,3151.395137,3151.186811,3150.569531,3147.927163,3150.072686,...,3149.574419,3148.351541,3150.542540,3149.083086,3150.228688,3150.214074,3150.291113,3152.977776,3149.666599,3149.340378
33,3147.198904,3154.277472,3148.029873,3148.078164,3152.125590,3151.796724,3151.588365,3150.971076,3148.328403,3150.474107,...,3149.975649,3148.753021,3150.943894,3149.484624,3150.630326,3150.615696,3150.692740,3153.379605,3150.067787,3149.741525
36,3147.666261,3154.745160,3148.497021,3148.545288,3152.593424,3152.264505,3152.056101,3151.438801,3148.795721,3150.941667,...,3150.442954,3149.220659,3151.411364,3149.952340,3151.098173,3151.083523,3151.160574,3153.847709,3150.535037,3150.208720
32,3147.666815,3154.745702,3148.497583,3148.545850,3152.593961,3152.265044,3152.056641,3151.439342,3148.796277,3150.942213,...,3150.443510,3149.221203,3151.411914,3149.952880,3151.098709,3151.084059,3151.161111,3153.848235,3150.535595,3150.209280


In [39]:
df_dist.nlargest(5, 0)

,0,1,2,3,4,5,6,7,8,9,...,5853,5854,5855,5856,5857,5858,5859,5860,5861,5862
69,3173.740890,3180.837252,3174.560198,3174.607119,3178.693696,3178.361916,3178.151078,3177.533222,3174.868156,3177.027088,...,3176.514507,3175.310547,3177.491861,3176.046364,3177.199308,3177.183528,3177.260973,3179.962577,3176.603548,3176.274274
0,3173.604229,3180.700452,3174.423634,3174.470566,3178.556826,3178.225068,3178.014251,3177.396399,3174.731515,3176.890340,...,3176.377874,3175.173760,3177.355154,3175.909544,3177.062429,3177.046658,3177.124100,3179.825585,3176.466941,3176.137692
68,3173.519780,3180.615982,3174.339199,3174.386133,3178.472346,3178.140592,3177.929778,3177.311926,3174.647069,3176.805878,...,3176.293429,3175.089293,3177.270699,3175.825071,3176.977948,3176.962179,3177.039620,3179.741088,3176.382500,3176.053254
1,3172.062546,3179.158351,3172.882236,3172.929202,3177.014518,3176.682831,3176.472073,3175.854233,3173.189889,3175.348397,...,3174.836273,3173.631702,3175.813334,3174.367385,3175.520096,3175.504354,3175.581786,3178.282919,3174.925416,3174.596239
67,3172.020133,3179.115943,3172.839819,3172.886784,3176.972113,3176.640425,3176.429667,3175.811827,3173.147474,3175.305987,...,3174.793859,3173.589294,3175.770922,3174.324978,3175.477692,3175.461949,3175.539381,3178.240519,3174.883000,3174.553822


In [52]:
distance_cal(tmc.loc[tmc['tmc']=='110-04111','end_latitude'],tmc.loc[tmc['tmc']=='110-04111','end_longitude'],tmc.loc[tmc['tmc']=='110N04103','end_latitude'],tmc.loc[tmc['tmc']=='110N04103','end_longitude'])

15.912283228698993

In [60]:
from geopy.distance import geodesic

coords_1 = (tmc.loc[tmc['tmc']=='110-04111','end_latitude'].values, tmc.loc[tmc['tmc']=='110-04111','end_longitude'].values)
coords_2 = (tmc.loc[tmc['tmc']=='110N04103','end_latitude'].values,tmc.loc[tmc['tmc']=='110N04103','end_longitude'].values)

print(geodesic(coords_1, coords_2).km)

15.899577863172917


In [7]:
import MapMatching4GMNS as mmg

To avoid complex data folder settings, please always first put the input data on the current directory.


OSError: /usr/local/home/ysx28/Desktop/GMNS/tmc2gmns/TMC2GMNS/bin/MapMatching4GMNS.so: cannot open shared object file: No such file or directory